In [3]:
import os
import nltk
import math
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# 遍历获取文件夹下的文件路径
def traverse(root):
    pathDir = os.listdir(root)
    childs = []
    for allDir in pathDir:
        childs.append(os.path.join('%s%s' % (root, allDir)))
    return childs

# 小写，去标点，分词，去停用词，词性还原  返回单词
def get_words(text, del_type):
    lowers = text.lower()
    #remove the punctuation using the character deletion step of translate
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    no_punctuation = lowers.translate(remove_punctuation_map)
    words = nltk.word_tokenize(no_punctuation)
    filtered = [w for w in words if w not in stopwords.words('english')]
    word_tags = nltk.pos_tag(filtered)
    final =[]
    lemmatizer = WordNetLemmatizer()
    # 新句子
    for word_tag in word_tags:
        if word_tag[1] in del_type:
            # 去除助词，副词，形容词
            continue
        else:
            # 对于其他单词，还原后，加入到新的句子中
            final.append(lemmatizer.lemmatize(word_tag[0]))
    return final

### 跟目录要改
root_dir ='./clusters_kmeans_wizabstract/'
# https://blog.csdn.net/qq_27231343/article/details/51934490
del_type = ['CD','DT','IN','LS','PRP','RB','RBR','RBS','UH','RP','PRP','MD','EX']
childs = traverse(root_dir)
out = open(root_dir.strip('/.')+'_labels.txt','w')
corpus = []
for child in childs:
    #out.write("In cluster %s:\n"%re.findall('[0-9][0-9]*',child)[0])
    with open(child,'r',encoding='utf-8') as reader:
        # corpus = ''
        line = reader.readline()
        while line:
            line_dic = eval(line)
            title = line_dic['title']+' '
            # version1.0 缺失值处理方法：全部舍弃
            abstract = ''
            if 'abstract' in line_dic.keys():
                abstract = line_dic['abstract']+' '
            corpus.append(title+abstract)
            line = reader.readline()
#pre_words = get_words(corpus, del_type)
#count = Counter(pre_words)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(corpus)

In [6]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 5
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [9]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

n_top_words = 5
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)



Topic #0:
time web knowledge based process
Topic #1:
study using modeling language evaluation
Topic #2:
using based learning algorithm control
Topic #3:
analysis information data networks network
Topic #4:
based speech applications model method



In [1]:
import pyLDAvis
import pyLDAvis.sklearn
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data)

NameError: name 'lda' is not defined